# Leave-One-Subject-Out Evaluation

In [ ]:
import numpy as np
import cv2
import dlib
from imutils import face_utils
import glob
import pickle
from random import shuffle
from sklearn.model_selection import train_test_split
from sklearn import metrics
import json
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import yaml

### 3D Model points for SolvePnP
We will approximate the 3D points of the following face parts with the correspoinding coordinates. This is a general model of the human face and we do not need to worry much about absolute accuracy.

In [ ]:
def get_camera_parameters(size):
    focal_length = size[1]
    center = (size[1]/2, size[0]/2)
    camera_matrix = np.array(
                             [[focal_length, 0, center[0]],
                             [0, focal_length, center[1]],
                             [0, 0, 1]], dtype = "double"
                             )
    dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion
    
    return camera_matrix, dist_coeffs

In [ ]:
def get_full_image_points(landmarks):
    image_points = np.zeros((68, 2))

    for i in range(68):
        image_points[i, :] = (landmarks[i]['x'], landmarks[i]['y'])
    
    return image_points

In [ ]:
def get_full_model_points(filename='model_points.txt'):
    """Get all 68 3D model points from file"""
    raw_value = []
    with open(filename) as file:
        for line in file:
            raw_value.append(line)
    model_points = np.array(raw_value, dtype=np.float32)
    model_points = np.reshape(model_points, (3, -1)).T

    # Transform the model into a front view.
    model_points[:, 2] *= -1

    return model_points

In [ ]:
def visualize_image(im, rotation_vector, translation_vector, image_points, camera_matrix):
    for point in image_points:
        cv2.circle(im, (int(point[0]), int(point[1])), 3, (0, 0, 255), -1)
    cv2.circle(im, (int(iris_left[0]), int(iris_left[1])), 3, (0, 0, 255), -1)
    cv2.circle(im, (int(iris_right[0]), int(iris_right[1])), 3, (0, 0, 255), -1)
    
    # Project the 3D point (0.55592, 6.5629, 300.0) onto the image plane.
    # We use this to draw a line sticking out of the nose
    (nose_end_point2D, jacobian) = cv2.projectPoints(
        np.array([(0.55592, 6.5629, 300.0)]), rotation_vector, translation_vector, camera_matrix, dist_coeffs
        )
    # Draw a line connecting the two points. This line must show
    # the direction out of the nose
    p1 = ( int(image_points[33][0]), int(image_points[33][1]) )
    p2 = ( int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]) )
    cv2.line(im, p1, p2, (255,0,0), 2)
    
    # Display image
    cv2.imshow(output, im)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

***

In [ ]:
participants = glob.glob('dataset/*')
print(len(participants))

In [ ]:
# Number of training examples to use(0-2758)
DATASET_SIZE = 2728
DEBUG = False

# Load the dataset
with open('data_cleaned.json') as json_file:
    data_all = json.load(json_file)
# Extract the keys in sorted order
keys_all = sorted(data_all)
# Convert python list to np array
keys_all = np.asarray(keys_all)
print(len(keys_all))

In [ ]:
# Accuracy metrics for the whole dataset. These are compouter
# by leaving every Subject out one time, calculating the accuracy for each
# one and then taking the mean.
accuracy_rf_total = 0
accuracy_svm_total = 0
precision_rf_total = 0
precision_svm_total = 0
recall_rf_total = 0
recall_svm_total = 0

model_points = get_full_model_points()

# Array to keep track of subjects with low score
low_score_subjects_rf = []
low_score_subjects_svm = []

for j in range(len(participants)):
    uuid_excluded = participants[j].split('/')[1]

    indices_excluded = []
    keys_excluded = []
    for i in range(DATASET_SIZE):
        key = keys_all[i]
        uuid = key.split('/')[0]
        if(uuid == uuid_excluded):
            indices_excluded.append(i)
            keys_excluded.append(key)
    keys = np.delete(keys_all, indices_excluded)

    CURRENT_DATASET_SIZE = keys.shape[0]

    X = np.zeros((CURRENT_DATASET_SIZE, 14, 1))
    y = np.zeros(CURRENT_DATASET_SIZE)

    # Indices that the SolvePnP failed
    failed_indices = []

    for i in range(CURRENT_DATASET_SIZE):
        key = keys[i]

        # Approximate camera intrinsic parameters
#         im = cv2.imread('dataset/' + key)   # This imread is time consuming! Another way?
#         size = im.shape
        with open('dataset/' + key.split('/')[0] + '/data.yml', 'r') as stream:
            try:
                laptop = yaml.safe_load(stream)['laptop']
                if(laptop == 'k' or laptop == 'K'):
                    size = (480, 640, 3)
                elif(laptop == 'c' or laptop == 'C'):
                    size = (720, 1280, 3)
            except yaml.YAMLError as exc:
                print(exc)

        landmarks = data_all[key]['landmarks']
        
        camera_matrix, dist_coeffs = get_camera_parameters(size)
        
        image_points = get_full_image_points(landmarks)

        # Solve the PnP problem with the parameters specified above
        # and obtain rotation and translation vectors
        (success, rotation_vector, translation_vector) = cv2.solvePnP(
            model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE
            )
        
        # Data from Architecture #2. Reshape is done for compatibility reasons
        iris_right = np.reshape(np.asarray(data_all[key]['iris_right']), (2, 1))
        iris_left = np.reshape(np.asarray(data_all[key]['iris_left']), (2, 1))

        # Data from Architecture #3
        left_vector = np.asarray( (abs(iris_left[0] - landmarks[39]['x']), abs(iris_left[1] - landmarks[39]['y'])) )
        right_vector = np.asarray( (abs(iris_right[0] - landmarks[42]['x']), abs(iris_right[1] - landmarks[42]['y'])) )

        X[i, :] = np.concatenate((rotation_vector, translation_vector, iris_left, iris_right, 
                                 left_vector, right_vector), axis=0)

        # Check if it is positive or negative example
        output = key.split('/')[1]
        if(output == 'positive'):
            y[i] = 1
        elif(output == 'negative'):
            y[i] = 0
    
        # Remove examples that SolvePnP crashed
        if(X[i, 0] > 10000):
            print(key)
            failed_indices.append(i)
    X = np.delete(X, failed_indices, axis=0)
    y = np.delete(y, failed_indices, axis=0)
    
    X = X.squeeze()

    m = X.mean(axis=0)
    std = X.std(axis=0)
    
    X_scaled = (X - m)/std
    
    ### Train and Predict
    X_train, y_train = X_scaled, y

    rf_classifier = RandomForestClassifier(n_estimators=500, random_state=1)
    rf_classifier.fit(X_train, y_train)

    svm_classifier = svm.SVC(C=10, kernel='rbf', gamma='scale', probability=True)
    svm_classifier.fit(X_train, y_train)

#     with open('classifiers/rf/' + uuid_excluded + '.pickle', 'wb') as f:
#         pickle.dump(rf_classifier, f, pickle.HIGHEST_PROTOCOL)
#     with open('classifiers/svm/' + uuid_excluded + '.pickle', 'wb') as f:
#         pickle.dump(svm_classifier, f, pickle.HIGHEST_PROTOCOL)

# # Retrieve the saved classifiers
#     with open('classifiers/rf/' + uuid_excluded + '.pickle', 'rb') as f:
#         rf_classifier = pickle.load(f)
#     with open('classifiers/svm/' + uuid_excluded + '.pickle', 'rb') as f:
#         svm_classifier = pickle.load(f)

# Construct the validation dataset
    X_eval = np.zeros((len(keys_excluded), 14, 1))
    y_eval = np.zeros(len(keys_excluded))

    for i in range(len(keys_excluded)):
        key = keys_excluded[i]

#         im = cv2.imread('dataset/' + key)   # This imread is time consuming! Another way?
#         size = im.shape
        with open('dataset/' + key.split('/')[0] + '/data.yml', 'r') as stream:
            try:
                laptop = yaml.safe_load(stream)['laptop']
                if(laptop == 'k' or laptop == 'K'):
                    size = (480, 640, 3)
                elif(laptop == 'c' or laptop == 'C'):
                    size = (720, 1280, 3)
                else:
                    print('Failed to load size')
            except yaml.YAMLError as exc:
                print(exc)

        landmarks = data_all[key]['landmarks']
        
        camera_matrix, dist_coeffs = get_camera_parameters(size)
        
        image_points = get_full_image_points(landmarks)

        # Solve the PnP problem with the parameters specified above
        # and obtain rotation and translation vectors
        (success, rotation_vector, translation_vector) = cv2.solvePnP(
            model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE
            )

        # Data from Architecture #2. Reshape is done for compatibility reasons
        iris_right = np.reshape(np.asarray(data_all[key]['iris_right']), (2, 1))
        iris_left = np.reshape(np.asarray(data_all[key]['iris_left']), (2, 1))

        # Data from Architecture #3
        left_vector = np.asarray( (abs(iris_left[0] - landmarks[39]['x']), abs(iris_left[1] - landmarks[39]['y'])) )
        right_vector = np.asarray( (abs(iris_right[0] - landmarks[42]['x']), abs(iris_right[1] - landmarks[42]['y'])) )

        X_eval[i, :] = np.concatenate((rotation_vector, translation_vector, iris_left, iris_right, 
                                      left_vector, right_vector), axis=0)
        
        # Check if it is positive or negative example
        output = key.split('/')[1]
        if(output == 'positive'):
            y_eval[i] = 1
        elif(output == 'negative'):
            y_eval[i] = 0

    X_eval = X_eval.squeeze()

    # Normalization
    m_eval = X_eval.mean(axis=0)
    std_eval = X_eval.std(axis=0)
    X_eval = (X_eval - m_eval)/std_eval

    # Predict Random Forest
    y_pred_rf = rf_classifier.predict(X_eval)
    rf_accuracy_subject = metrics.accuracy_score(y_eval, y_pred_rf)

    # Predict SVM
    threshold = 0.3
    y_prob_svm = svm_classifier.predict_proba(X_eval)
    y_pred_svm = (y_prob_svm[:, 1] >= threshold).astype(int)
    svm_accuracy_subject = metrics.accuracy_score(y_eval, y_pred_svm)
    
    confusion_matrix_rf = metrics.confusion_matrix(y_eval, y_pred_rf)
    confusion_matrix_svm = metrics.confusion_matrix(y_eval, y_pred_svm)
    
    precision_rf = confusion_matrix_rf[1][1]/(confusion_matrix_rf[1][1] + confusion_matrix_rf[0][1])
    recall_rf = confusion_matrix_rf[1][1]/(confusion_matrix_rf[1][1] + confusion_matrix_rf[1][0])
    precision_svm = confusion_matrix_svm[1][1]/(confusion_matrix_svm[1][1] + confusion_matrix_svm[0][1])
    recall_svm = confusion_matrix_svm[1][1]/(confusion_matrix_svm[1][1] + confusion_matrix_svm[1][0])
    
    print('RF  #{} Accuracy: {} | Precision: {} | Recall: {}'.format(j, round(rf_accuracy_subject,3),
                                                            round(precision_rf,2), round(recall_rf, 2)))
    print('SVM #{} Accuracy: {} | Precision: {} | Recall: {}'.format(j, round(svm_accuracy_subject, 3),
                                                            round(precision_svm, 2), round(recall_svm, 2)))

    if(rf_accuracy_subject <= 0.5):
        low_score_subjects_rf.append(uuid_excluded)
        with open('classifiers/rf/' + uuid_excluded + '.pickle', 'wb') as f:
            pickle.dump(rf_classifier, f, pickle.HIGHEST_PROTOCOL)
    if(svm_accuracy_subject <= 0.5):
        low_score_subjects_svm.append(uuid_excluded)
        with open('classifiers/svm/' + uuid_excluded + '.pickle', 'wb') as f:
            pickle.dump(svm_classifier, f, pickle.HIGHEST_PROTOCOL)
    
    accuracy_rf_total += rf_accuracy_subject*len(keys_excluded)
    accuracy_svm_total += svm_accuracy_subject*len(keys_excluded)
    precision_rf_total += precision_rf*len(keys_excluded)
    precision_svm_total += precision_svm*len(keys_excluded)
    recall_rf_total += recall_rf*len(keys_excluded)
    recall_svm_total += recall_svm*len(keys_excluded)
    
accuracy_rf_total /= DATASET_SIZE
accuracy_svm_total /= DATASET_SIZE
precision_rf_total /= DATASET_SIZE
precision_svm_total /= DATASET_SIZE
recall_rf_total /= DATASET_SIZE
recall_svm_total /= DATASET_SIZE

In [ ]:
print('Total Accuracy RF:  {}'.format(accuracy_rf_total))
print('Total Accucary SVM: {}'.format(accuracy_svm_total))

In [ ]:
print('Precision RF: {}'.format(precision_rf_total))
print('Precision SVM: {}'.format(precision_svm_total))

In [ ]:
print('Recall RF: {}'.format(recall_rf_total))
print('Recall SVM: {}'.format(recall_svm_total))

In [ ]:
print(len(low_score_subjects_rf))
print(len(low_score_subjects_svm))

In [ ]:
file1 = open("low_score_rf.txt", "w") 
for i in low_score_subjects_rf:
    file1.write(i + '\n')
    
file2 = open("low_score_svm.txt", "w")
for i in low_score_subjects_svm:
    file2.write(i + '\n')

In [ ]:
print(X_scaled[:, 0:14].shape)